## Plain bubble

In [ ]:
from PIL import Image, ImageDraw
import IPython.display
import math

def draw_bubble(center, b_radius, image):
    draw = ImageDraw.Draw(im)
    white = (255, 255, 255, 255)
    border = (93, 199, 241, 255)
    cyan = (150, 230, 255, 255)
    
    draw.ellipse((center[0]-b_radius, center[1]-b_radius, center[0] + b_radius, center[1] + b_radius), fill=cyan, outline=border)

    reflect_w = b_radius // 5
    n_spots = b_radius
    
    for w in range(reflect_w):
        reflect_r = int(b_radius*(0.65-w*0.01))
        for n in range(w, n_spots - w):
            angle = math.pi/3 - math.pi/3 * n/n_spots
            spot = [int(center[0] - reflect_r * math.sin(angle)), int(center[1] - reflect_r * math.cos(angle))] 
            draw.ellipse((spot[0] - 1, spot[1] - 1, spot[0] + 1, spot[1] + 1), fill=white)

    draw.ellipse((center[0]-b_radius, center[1]-b_radius, center[0] + b_radius, center[1] + b_radius), fill=None, outline=border)
    
b_radius = 50

im = Image.new('RGBA', (2*b_radius, 2*b_radius), (0, 0, 0, 50))
center = [b_radius, b_radius]
draw_bubble(center, b_radius, im)

IPython.display.display(im)

## Animated bubbles

In order to get a seamless background, we'll wrap everything so that we can use `background-repeat`.

In [ ]:
from random import random, randrange, seed
import math

seed(0)
images = []
width = 128
max_frame = width

# Possible bubble trajectories
trajectories = []
for i in range(2, 4):
    div = 2**i
    traj =  [1] * (max_frame//div)
    while len(traj) < max_frame:
        traj +=  [-traj[-1]] * (max_frame//div)
    assert(traj.count(1) == traj.count(-1))
    trajectories.append(traj)
    
trajectories.append([0] * max_frame)
    
# Bubble characteristics
n_bubbles = 10
centers = [[randrange(0, width), randrange(0, width)] for i in range(n_bubbles)]
b_radii = [randrange(2, 12) for i in range(n_bubbles)]
b_offset = [3.14*2*random() for i in range(n_bubbles)]
b_offset = [randrange(0, max_frame) for i in range(n_bubbles)]
b_traj = [trajectories[randrange(0, len(trajectories))] for i in range(n_bubbles)]

for im_id in range(0, max_frame):
    im = Image.new('RGBA', (width, width), (0, 0, 0, 255))
    draw = ImageDraw.Draw(im)
    
    for center_id in range(len(centers)):
        center = centers[center_id]
        b_radius = b_radii[center_id]
        traj = b_traj[center_id]
        draw_bubble(center, b_radius, im)
                
            
        # wrap: if close to border, also draw bubble on the other side
        if center[0] + b_radius > width :
            draw_bubble([center[0] - width, center[1]], b_radius, im)
        if center[0] - b_radius < 0:
            draw_bubble([center[0] + width, center[1]], b_radius, im)
        if center[1] + b_radius > width :
            draw_bubble([center[0], center[1] - width], b_radius, im)
        if center[1] - b_radius < 0 :            
            draw_bubble([center[0], center[1] + width], b_radius, im)
    
        # Bubble movement
        center[1] -= 1
        center[0] += traj[(im_id + b_offset[center_id]) % max_frame]
        
        # wrap: bring center back in [0, width]
        if center[0] > width:
            center[0] -= width
        if center[0] < 0:
            center[0] += width
        if center[1] > width:
            center[1] -= width
        if center[1] < 0:
            center[1] += width
              
    images.append(im)

images[0].save('../images/piltest.gif',
               save_all=True, 
               append_images=images[1:], 
               optimize=False, 
               disposal=2, 
               duration=1000/20, # duration = 1/fps, in ms
               loop=0, 
               transparency=0)


## Top of the glass

Some fresh-looking water.

In [ ]:

pixel_size = 10
increments = 5
width, height = (1900//pixel_size, 1080//pixel_size)
im = Image.new('RGB', (width, height), (93, 199, 241))

pixels = im.load()

for step in range(25):
    for i in range(width):
        for j in range(height):
            if random() > j/height:
                r, g, b = pixels[i,j]
                if random() > 0.5:
                    r = min(r + increments, 255)
                    
                if random() > 0.5:
                    g = min(g + increments, 255)
                
                if random() > 0.5:
                    b = min(b + increments, 255)
                pixels[i,j] = (r, g, b)


IPython.display.display(im)

In [ ]:
im.save('testcoolwater.jpg')